In [1]:
import os
import cv2
# import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

import imageio
import imgaug as ia
import imgaug.augmenters as iaa

# from moviepy.editor import *
# %matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img

from sklearn.model_selection import train_test_split
import tensorflow.keras
from tensorflow.keras.layers import*
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model




In [4]:

DATASET_DIR  = 'E:/20_exercise'
CLASSES_LIST = ["pushups", "squat", "suryanamaskar" ]
SEQUENCE_LENGTH = 500
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64


seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

# DATASET_DIR  = "E:\\20_exercise"
# CLASSES_LIST = ["pushups", "squat", "suryanamaskar" ]
# SEQUENCE_LENGTH = 250
# IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
import os
DIR = DATASET_DIR
import cv2
import random
import numpy as np
# img = cv2.imread('E:\\PHOTOS\\Screenshots\\panda.png')
def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

def brightness(img, low, high):
    value = random.uniform(low, high)
    img_float32 = np.float32(img)

    hsv = cv2.cvtColor(img_float32, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

    


# ----------------------------------------------------------------------------------------------------------------------------
#############################################################################################################################
#############################################################################################################################
# ----------------------------------------------------------------------------------------------------------------------------
def frame_nomalize(img):
    resized_frame = cv2.resize(img, (64, 64))
    normalized_frame = resized_frame / 255
    return normalized_frame


def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []
    
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video. 
        success, frame = video_reader.read() 
        

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        normalized_frame = frame_nomalize(frame)
        frames_list.append(normalized_frame)
    
    # Release the VideoCapture object. 
    video_reader.release()
#     print(l1)
#     print(l2)

    # Return the frames list.
    return frames_list




# ----------------------------------------------------------------------------------------------------------------------------
#############################################################################################################################
#############################################################################################################################
# ----------------------------------------------------------------------------------------------------------------------------


# CREATE DATASET WITH DATA AUGMENTATION

# ----------------------------------------------------------------------------------------------------------------------------
#############################################################################################################################
#############################################################################################################################
# ----------------------------------------------------------------------------------------------------------------------------


def create_dataset2():

    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            print(file_name)
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            
            frames_1 = []
            frames_2 = []
            frames_3 = []
            frames_4 = []
            frames_5 = []


            for i in frames:
                
# ROTATE
                rotate=iaa.Affine(rotate=(-50, 30))
                rotated_image=rotate.augment_image(i)
#                 nf = frame_nomalize(rotated_image)
#                 frames_1.append(nf)
                
                
#                 img = i.astype(np.uint8)
#                 gaussian_noise=iaa.AdditiveGaussianNoise(10,20)
#                 noise_image=gaussian_noise.augment_image(i)
#                 nf = frame_nomalize(noise_image)
#                 frames_2.append(nf)
                
#               crop  
                crop = iaa.Crop(percent=(0, 0.3)) # crop image
                corp_image=crop.augment_image(rotated_image)
                nf = frame_nomalize(corp_image)
                frames_1.append(nf)
                
#                 flip
                flip_hr=iaa.Fliplr(p=1.0)
                flip_hr_image= flip_hr.augment_image(i)
#                 nf = frame_nomalize(flip_hr_image)
#                 frames_4.append(nf)

#                 contrast
                contrast=iaa.GammaContrast(gamma=2.0)
                contrast_image =contrast.augment_image(flip_hr_image)
                nf = frame_nomalize(contrast_image)
                frames_2.append(nf)




            

            

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

                features.append(frames_1)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

                features.append(frames_2)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

#                 features.append(frames_3)
#                 labels.append(class_index)
#                 video_files_paths.append(video_file_path)

#                 features.append(frames_4)
#                 labels.append(class_index)
#                 video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

    
features, labels, video_files_paths = create_dataset2()

# print(features[0])
print("DONE")

Extracting Data of Class: pushups
pushup_1.mp4
pushup_10.mp4
pushup_11.mp4
pushup_12.mp4
pushup_13.mp4
pushup_14.mp4
pushup_15.mp4
pushup_2.mp4
pushup_3.mp4
pushup_4.mp4
pushup_5.mp4
pushup_6.mp4
pushup_7.mp4
pushup_8.mp4
pushup_9.mp4
Extracting Data of Class: squat
squat_10.mp4
squat_11.mp4
squat_12.mp4
squat_13.mp4
squat_14.mp4
squat_15.mp4
squat_2.mp4
squat_3.mp4
squat_4.mp4
squat_5.mp4
squat_6.mp4
squat_7.mp4
squat_8.mp4
squat_9.mp4
Extracting Data of Class: suryanamaskar
suryanamaskar_1.mp4
suryanamaskar_10.mp4
suryanamaskar_11.mp4
suryanamaskar_12.mp4
suryanamaskar_13.mp4
suryanamaskar_14.mp4
suryanamaskar_15.mp4
suryanamaskar_2.mp4
suryanamaskar_3.mp4
suryanamaskar_4.mp4
suryanamaskar_5.mp4
suryanamaskar_6.mp4
suryanamaskar_7.mp4
suryanamaskar_8.mp4
suryanamaskar_9.mp4
DONE


In [5]:
# print((features[0].size))
print(features.size)
print(features.ndim)
print(labels)
len(labels)

811008000
5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


132

In [77]:
print((features[0].size))
print(features.size)
print(features.ndim)


1228800
3686400
5


sbdjvae


start


In [6]:
print("start")
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
print("MIDDLE")
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.20, shuffle = True,
                                                                            random_state = seed_constant)


print("DONE")

start
MIDDLE
DONE


In [7]:


def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(8, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Dropout(0.2)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [8]:
LRCN_model = create_LRCN_model()
EPOCHS = 10
# Display the success message.
print("Model Created Successfully!")
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
# LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 4 ,
#                                              shuffle = True, validation_split = 0.2)
LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4 ,
                                             shuffle = True, validation_split = 0.2)

print('\n\nTASK COMPLETED')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 500, 64, 64, 8)   224       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 500, 16, 16, 8)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 500, 16, 16, 8)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 500, 16, 16, 16)  1168      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 500, 4, 4, 16)    0

21/21 [==============================] - 28s 1s/step - loss: 0.7748 - accuracy: 0.5357 - val_loss: 1.9005 - val_accuracy: 0.4286
Epoch 38/50
21/21 [==============================] - 28s 1s/step - loss: 0.7744 - accuracy: 0.5357 - val_loss: 1.9449 - val_accuracy: 0.4286
Epoch 39/50
21/21 [==============================] - 28s 1s/step - loss: 0.7750 - accuracy: 0.5357 - val_loss: 1.9685 - val_accuracy: 0.4286
Epoch 40/50
21/21 [==============================] - 28s 1s/step - loss: 0.7735 - accuracy: 0.5357 - val_loss: 1.9746 - val_accuracy: 0.4286
Epoch 41/50
21/21 [==============================] - 29s 1s/step - loss: 0.7742 - accuracy: 0.4881 - val_loss: 1.9966 - val_accuracy: 0.1905
Epoch 42/50
21/21 [==============================] - 28s 1s/step - loss: 0.7764 - accuracy: 0.4881 - val_loss: 2.0181 - val_accuracy: 0.4286
Epoch 43/50
21/21 [==============================] - 28s 1s/step - loss: 0.7748 - accuracy: 0.5357 - val_loss: 2.0334 - val_accuracy: 0.4286
Epoch 44/50
21/21 [======

In [9]:
# save  model
from tensorflow.keras.models import Sequential,load_model
# test and save  model accuracy
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
    
model_file_name = "E:\\test_output\\augmentation_final__model_500_15_(aug_2).h5"

# Save the Model.
LRCN_model.save(model_file_name)
new_model = load_model(model_file_name)
print("model is saved")

1/1 [==============================] - 121s 121s/step - loss: 2.1719 - accuracy: 0.2593
model is saved


In [8]:
print("sn ")

sn 


In [187]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH):
    from tensorflow.keras.models import load_model

#     model_file_name = "E:\\test_output\\final__model.h5"
    model_file_name = "E:\\test_output\\augmentation_final__model_250_15.h5"

    new_model = load_model(model_file_name)

    '''
    This function will perform action recognition on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    output_file_path: The path where the ouput video with the predicted action being performed overlayed will be stored.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''
    
    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a queue to store video frames.
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Iterate until the video is accessed successfully.
    while video_reader.isOpened():

        # Read the frame.
        ok, frame = video_reader.read() 
        
        # Check if frame is not read properly then break the loop.
        if not ok:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list.
        frames_queue.append(normalized_frame)

        # Check if the number of frames in the queue are equal to the fixed sequence length.
        if len(frames_queue) == SEQUENCE_LENGTH:

            # Pass the normalized frames to the model and get the predicted probabilities.
            predicted_labels_probabilities = new_model.predict(np.expand_dims(frames_queue, axis = 0))[0]


            # Get the index of class with highest probability.
            predicted_label = np.argmax(predicted_labels_probabilities)
#             print("predicted_labels_probabilities : ",predicted_label)


            # Get the class name using the retrieved index.
            predicted_class_name = CLASSES_LIST[predicted_label]

    video_reader.release()
    return predicted_class_name
        # Write predicted class name on top of the frame.
#         cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#         # Write The frame into the disk using the VideoWriter Object.
#         video_writer.write(frame)
        
#     # Release the VideoCapture and VideoWriter objects.
#      video_writer.release()





# ##########################################################33

import cv2
import collections
from collections import *
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

SEQUENCE_LENGTH = 250
# Construct the output video path.
output_video_file_path = "E:\\test_output\\o1.mp4"
path = str(input("Enter path : "))
# Perform Action Recognition on the Test Video.
exercise = predict_on_video(path, output_video_file_path, SEQUENCE_LENGTH)

# Display the output video.
# VideoFileClip(output_video_file_path, audio=False, target_resolution=(300,None)).ipython_display()

Enter path : E:\\test_output\\sn2.mp4
1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 156ms/step


1/1 [==============================] - 0s 150ms/step


In [188]:
exercise 

'suryanamaskar'

In [189]:
#------------------------------------------------- COUNTER--------------------------------------------------------------

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


def calculate_angle_squat(a,b,c):
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)

        if angle >180.0:
            angle = 360-angle

        return angle 
    
# ##############################################################################################################################
# ##############################################################################################################################

def count_squat(exercise, path):
    # render angel in vedio
    # squat
    # final counter
    import cv2
    stage = "up"

    

    # ///////////////////////
    cap = cv2.VideoCapture(path)
#     cap = cv2.VideoCapture("C:\\Users\\abc-pc\\Downloads\\Exercise\\squat\\video_squat_16.mp4")
    counter = 0 
    stage = None

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark


                l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]



                # Calculate angle
                angle_l = calculate_angle_squat(l_hip, l_knee, l_ankle)
                angle_r = calculate_angle_squat(r_hip, r_knee, r_ankle)

    #             print("Left : ",angle_l)
    #             print("Right : ",angle_r)

    #             angle = str(angle_l) +" : "+ str(angle_r)
                angle_up = min(angle_l,angle_r)
    #             print(angle)
    
#     count set
                if angle_up > 160:
                    stage = "up"
                if angle_up < 150 and stage == "up":
                    stage="down"
                    counter +=1
                    print(counter)
                info = "Set : " + str(counter)
                e = exercise


                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 1
                color = (255, 0, 0)
                thickness = 2
                image = cv2.putText(image, info, [100,100], font, fontScale, color, thickness, cv2.LINE_AA)
                image = cv2.putText(image, e, [100,200], font, fontScale, color, thickness, cv2.LINE_AA)


            except:
                pass


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(0,66,230), thickness=2, circle_radius=2) 
                                     )               

            b = cv2.resize(image,(600,600))
            cv2.imshow('Mediapipe Feed', b)
            

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# ##############################################################################################################################
# ##############################################################################################################################

        
def count_pushup(exercise, path):
    # render angel in vedio
    # pushup
    # final counter

    stage = "up"
    l = []
    def calculate_angle_squat(a,b,c):
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)

        if angle >180.0:
            angle = 360-angle

        return angle 


    # ///////////////////////
#     cap = cv2.VideoCapture("C:\\Users\\abc-pc\\Downloads\\Exercise\\pushup\\video_pushup_12.mp4")
    cap = cv2.VideoCapture(path)
    counter = 0 
    stage = None

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark


                l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]



                # Calculate angle
                angle_l = calculate_angle_squat(l_shoulder, l_elbow, l_wrist)
                angle_r = calculate_angle_squat(r_shoulder, r_elbow, r_wrist)

    # #             print("Left : ",angle_l)
    # #             print("Right : ",angle_r)

    #             angle = str(angle_l) +" : "+ str(angle_r)
                angle_up = min(angle_l,angle_r)
    #             print(angle_up)
                if angle_up > 150:
                    stage = "up"
                if angle_up < 100 and stage == "up":
                    stage="down"
                    counter +=1
                    
                info = "Set : " + str(counter)
                e = exercise


                l.append(angle_up)


                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 3
                color = (255, 0, 0)
                thickness = 3
                image = cv2.putText(image, info, [100,100], font, fontScale, color, thickness, cv2.LINE_AA)
                image = cv2.putText(image, e, [100,200], font, fontScale, color, thickness, cv2.LINE_AA)


            except:
                pass


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(0,66,230), thickness=2, circle_radius=2) 
                                     )               

            b = cv2.resize(image,(500,500))
            cv2.imshow('Mediapipe Feed', b)
            


            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

#     print("min_angle",min(l))
#     print("max_angle",max(l))
    
# count_squat()
# count_pushup()


def calculate_angle_squat(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


# ///////////////////////
def count_suryanamaskar(exercise, path):
    # render angel in vedio
    # squat
    # final counter
    import cv2
    stage = "up"

    

    # ///////////////////////
    cap = cv2.VideoCapture(path)
#     cap = cv2.VideoCapture("C:\\Users\\abc-pc\\Downloads\\Exercise\\squat\\video_squat_16.mp4")
    counter = 0 
    stage = None

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark


                l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]


                l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]


                r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                wha = calculate_angle_squat(l_wrist, l_hip, l_ankle)
                sha = calculate_angle_squat(l_shoulder, l_hip, l_ankle)
#                 wha = format(wha,".0f")
#                 sha = format(sha,".0f")


                if  wha >=165 and stage == "up":
                    stage = "down"
                if  wha >=165 and stage == "down":
                    stage = "up"
                    counter += 1

                info = "Set : " + str(counter)
                e = exercise



                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 1
                color = (255, 0, 0)
                thickness = 2
                image = cv2.putText(image, info, [100,50], font, fontScale, color, thickness, cv2.LINE_AA)
                image = cv2.putText(image, e, [10,100], font, fontScale, color, thickness, cv2.LINE_AA)


            except:
                pass


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(0,66,230), thickness=2, circle_radius=2) 
                                     )               

            b = cv2.resize(image,(600,600))
            cv2.imshow('Mediapipe Feed', b)
            

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# ##############################################################################################################################
# ##############################################################################################################################

if exercise == "squat":
    count_squat(exercise,path)
if exercise == "pushups":
    count_pushup(exercise,path)    

if exercise == "suryanamaskar":
    count_suryanamaskar(exercise,path)    


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
